# Numerical evaluation of second Renyi entropy in Z2 LGT using gauged PEPS transfer operator method

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.linalg import schur, eigvals, eig
from itertools import product

## Basic elements: transfer operator matrix $\tau_0$ and boundary operator $X$

In [2]:
# Parameters:
alpha = 1.0
beta  = 0.1
gamma = 0.0
delta = 0.95

In [3]:
tau0 = np.array([[np.abs(alpha)**2, np.abs(gamma)**2, 0, 0],
                 [np.abs(gamma)**2, np.abs(delta)**2, 0, 0],
                 [0               , 0               , np.abs(beta)**2, np.abs(beta)**2],
                 [0               , 0               , np.abs(beta)**2, np.abs(beta)**2]
                ])
tau0

array([[1.    , 0.    , 0.    , 0.    ],
       [0.    , 0.9025, 0.    , 0.    ],
       [0.    , 0.    , 0.01  , 0.01  ],
       [0.    , 0.    , 0.01  , 0.01  ]])

In [4]:
# a Schur decomposition realizes (71) for a real block-diagonal tau0 into orthogonal V and diagonal L:
L,V = schur(tau0)
lambda_vals = np.diag(L)
print("l1...l4 num:    ",lambda_vals)

l1...l4 num:     [0.02   0.     1.     0.9025]


In [5]:
V

array([[ 0.        ,  0.        ,  1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ],
       [ 0.70710678, -0.70710678,  0.        ,  0.        ],
       [ 0.70710678,  0.70710678,  0.        ,  0.        ]])

In [6]:
# here identical to eigenvalue decomposition:
w,v = np.linalg.eig(tau0)
v

array([[ 0.        ,  0.        ,  1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ],
       [ 0.70710678, -0.70710678,  0.        ,  0.        ],
       [ 0.70710678,  0.70710678,  0.        ,  0.        ]])

In [7]:
np.allclose(V @ L @ V.T, tau0)

True

In [8]:
# We want the same ordering as in (126) and (127) to avoid expensive looping over lambda_4=0:
l1 = 0.5*(np.abs(alpha)**2+np.abs(delta)**2 + np.sqrt((np.abs(alpha)**2-np.abs(delta)**2)**2+4*np.abs(gamma)**4))
l2 = 0.5*(np.abs(alpha)**2+np.abs(delta)**2 - np.sqrt((np.abs(alpha)**2-np.abs(delta)**2)**2+4*np.abs(gamma)**4))
l3 = 2*np.abs(beta)**2
l4 = 0.0
print("l1...l4 analyt: ",[l1,l2,l3,l4])

cols = [list(V[:,i]) for i in range(4)] # all eigenvectors
V3 = [0,0, 1/np.sqrt(2),1/np.sqrt(2)]
V4 = [0,0,-1/np.sqrt(2),1/np.sqrt(2)]
V3_ind = cols.index(V3) # identify correct indices of V3, V4 from known analytical form ...
V4_ind = cols.index(V4) # ... (because l2==l4 might occur)

perm = [np.where(np.isclose(lambda_vals,l1))[0], np.where(np.isclose(lambda_vals,l2))[0], V3_ind, V4_ind]
print("perm possible: ",perm) # all possible permutations of eigenvalue/vector ordering

V_ordered = V.copy()
for j in range(2): # find correct eigenvector positions to first two eigenvalues
    if len(perm[j])>1:
        allowed_inds = [x for x in perm[j] if (x!=V3_ind and x!=V4_ind)] # exclude mu=3,4 indices
        if len(allowed_inds)>1: # take care of possible l1==l2
            Vj_ind = allowed_inds[j]
            V_ordered[:,j] = V[:,Vj_ind]
            perm[j] = Vj_ind
        else:
            Vj_ind = allowed_inds[0]
            V_ordered[:,j] = V[:,Vj_ind]
            perm[j] = Vj_ind
    else:
        Vj_ind = perm[j][0]
        V_ordered[:,j] = V[:,Vj_ind]
        perm[j] = Vj_ind
V_ordered[:,2] = V3
V_ordered[:,3] = V4
print("perm taken:    ",perm)
lambda_vals = np.array([l1,l2,l3,l4])
L, V = np.diag(lambda_vals), V_ordered

l1...l4 analyt:  [1.0, 0.9024999999999999, 0.020000000000000004, 0.0]
perm possible:  [array([2]), array([3]), 0, 1]
perm taken:     [2, 3, 0, 1]


In [9]:
V

array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.70710678, -0.70710678],
       [ 0.        ,  0.        ,  0.70710678,  0.70710678]])

In [10]:
# Checks:
print("all elems in perm:                ",all(x in [0,1,2,3] for x in perm))
print("Schur decomposition tau0=V*L*V.T: ",np.allclose(V @ L @ V.T, tau0))
print("V is orthogonal:                  ",np.allclose(V @ V.T,np.identity(4)))
print("V is correctly ordered:           ",V[2,0]==0 and V[2,1]==0 and V[2,2]==1/np.sqrt(2) and V[2,3]==-1/np.sqrt(2))

all elems in perm:                 True
Schur decomposition tau0=V*L*V.T:  True
V is orthogonal:                   True
V is correctly ordered:            True


In [11]:
# M_mu matrices constructed from V:
Pu = np.array([[1,0],[0,0]])
Pd = np.array([[0,0],[0,1]])
M1 = V[0,0]*Pu + V[1,0]*Pd
M2 = V[0,1]*Pu + V[1,1]*Pd
M3 = (1.0/np.sqrt(2))*np.array([[0, 1],[1,0]])
M4 = (1.0/np.sqrt(2))*np.array([[0,-1],[1,0]])
M = [M1,M2,M3,M4]

In [12]:
print("Tr[M1*M1.T] = ",np.trace(M1 @ M1.T)) # realizes (74)
print("Tr[M1*M2.T] = ",np.trace(M1 @ M2.T))

Tr[M1*M1.T] =  1.0
Tr[M1*M2.T] =  0.0


In [13]:
X = np.array([[1,0,0,0],
              [0,0,0,1],
              [0,0,1,0],
              [0,1,0,0]
             ])

In [14]:
np.allclose(X, X.T) # X is diagonal

True

## Functions to calculate all relevant transfer row operators: 

Function calculating the norm of $\rho$ by exponentiating the single transfer row as 
\begin{equation}
Tr_A[\rho_A] = Tr_A[Tr_B[\rho]] = Tr[\rho] = Tr[E^{(1) N_y}] ,
\end{equation}
where E$^{(1)}$ is the trace along a row, i.e.
\begin{equation}
E^{(1)} = Tr_{\text{row}}[\tau_0^{\otimes N_x}] = \sum_{\{\mu\}} (\lambda_{\mu_1} \cdot\ldots\cdot \lambda_{\mu_{N_x}}) Tr[M_{\mu_1} \cdot\ldots\cdot M_{\mu_{N_x}}] M_{\mu_1} \otimes\ldots\otimes M_{\mu_{N_x}},
\end{equation}
cf. eq. (75)

In [15]:
def single_transfer_row(Nx, lambdas, M):
    print("calculating E1 ...")
    # for practical calculations, assume a thin lattice of dimensions Nx<9 << Ny~100
    E1 = np.zeros((2**Nx,2**Nx))

    # At every lattice site x along a row with length Nx, 
    # the index mu can take effectively 3 different values mu=(0,1,2) [because lambda_4=0].
    # We sum up all the possible 3^Nx configurations contributing to E1:
    for config in product(range(3), repeat=Nx):
        mu_of_x = np.array(config) # = \mu(x)
        M_prod_mu = np.identity(2)
        lambda_prod_mu = 1.0
        
        # loop (product) along a row x:
        for x in range(Nx):
            lambda_prod_mu *= lambdas[mu_of_x[x]] # = \prod_x lambda_{\mu(x)}
            M_prod_mu = M_prod_mu @ M[mu_of_x[x]] # = \prod_x M_{\mu(x)}
        
        # calculate tensor product M_{\mu(x=1)} x...x M_{\mu(x=Nx)}
        # to get matrix form of E1:
        M_tensor_mu = np.kron(M[mu_of_x[0]], M[mu_of_x[1]])
        for x in range(2,Nx):
            M_tensor_mu = np.kron(M_tensor_mu, M[mu_of_x[x]])
            
        E1 = E1 + (lambda_prod_mu * np.trace(M_prod_mu) * M_tensor_mu)
    return E1

def norm_rho(Nx, Ny, lambdas, M):
    print("calculating norm ...")
    # full state rho = E1^(Ny):
    E1 = single_transfer_row(Nx,lambdas,M)
    rho = np.linalg.matrix_power(E1,Ny)    
    return np.trace(rho) # returns norm of state

In [16]:
nrm=norm_rho(4,100,lambda_vals,M)
nrm

calculating norm ...
calculating E1 ...


1.0000040007307887

Calculate the "double transfer row" $E^{(2)}=E^{(1)} \otimes E^{(1)}$ explicitly with suitable (site-wise) tensor product structure:

\begin{align}
E^{(2)} = \sum_{\{\mu\},\{\nu\}} &(\lambda_{\mu_1} \cdot\ldots\cdot \lambda_{\mu_{N_x}}) (\lambda_{\nu_1} \cdot\ldots\cdot \lambda_{\nu_{N_x}}) Tr[M_{\mu_1} \cdot\ldots\cdot M_{\mu_{N_x}}] Tr[M_{\nu_1} \cdot\ldots\cdot M_{\nu_{N_x}}] \\
&(M_{\mu_1} \otimes M_{\nu_1}) \otimes\ldots\otimes (M_{\mu_{N_x}} \otimes M_{\nu_{N_x}})
\end{align}

In [17]:
def double_transfer_row(Nx, lambdas, M):
    print("calculating E2 ...")
    E2 = np.zeros((2**(2*Nx),2**(2*Nx)))

    # At every lattice site x along the row with length Nx, 
    # the indices mu,nu can take 3 different values mu,nu=(0,1,2).
    # We sum up all the possible (3*3)^Nx configurations contributing to E2:
    for config in product(range(3),range(3), repeat=Nx):
        munu_of_x = np.array(config) # contains mu(x) at index 2x, nu(x) at 2x+1
        lambda_prod_mu = 1.0
        lambda_prod_nu = 1.0
        M_prod_mu = np.identity(2)
        M_prod_nu = np.identity(2)
        
        # loop (product) along a row x:
        for x in range(Nx):
            lambda_prod_mu *= lambdas[munu_of_x[2*x]]   # = \prod_x lambda_{\mu(x)}
            lambda_prod_nu *= lambdas[munu_of_x[2*x+1]] # = \prod_x lambda_{\nu(x)}
            M_prod_mu = M_prod_mu @ M[munu_of_x[2*x]]   # = \prod_x M_{\mu(x)}
            M_prod_nu = M_prod_nu @ M[munu_of_x[2*x+1]] # = \prod_x M_{\nu(x)}

        # calculate tensor product  (M_{mu_1} x M_{nu_1}) x...x (M_{mu_Nx} x M_{nu_Nx})
        # to get matrix form of E2:
        M_tensor = np.kron(M[munu_of_x[0]], M[munu_of_x[1]])
        for x in range(1,Nx):
            M_tensor = np.kron(M_tensor, np.kron(M[munu_of_x[2*x]], M[munu_of_x[2*x+1]]))
        
        E2 = E2 + (lambda_prod_mu*lambda_prod_nu * np.trace(M_prod_mu)*np.trace(M_prod_nu) * M_tensor)
    return E2

In [18]:
E2 = double_transfer_row(4, lambda_vals,M)

calculating E2 ...


In [19]:
E2.shape

(256, 256)

The "double transfer row" $E^{(2)}=E^{(1)} \otimes E^{(1)}$ could also be immediately constructed as a matrix, but then does not have the desired (site-wise) tensor product structure:

In [20]:
e1 = single_transfer_row(4,lambda_vals,M)
e2 = np.kron(e1,e1)

calculating E1 ...


In [21]:
np.allclose(e2,E2)

False

In [22]:
np.trace(e2) - np.trace(E2)

0.0

Calculate the boundary transfer row $E^{(2)}_{||}$ with two $X$ insertions:

\begin{align}
E^{(2)}_{||} = \sum_{\{\mu\},\{\nu\}} &(\lambda_{\mu_1} \cdot\ldots\cdot \lambda_{\mu_{N_x}}) (\lambda_{\nu_1} \cdot\ldots\cdot \lambda_{\nu_{N_x}}) 
Tr[X (M_{\mu_1} \otimes M_{\nu_1}) \cdot\ldots\cdot (M_{\mu_{R_1}} \otimes M_{\nu_{R_1}}) X (M_{\mu_{R_1+1}} \otimes M_{\nu_{R_1+1}}) \cdot\ldots\cdot (M_{\mu_{N_x}} \otimes M_{\nu_{N_x}})] \\
&(M_{\mu_1} \otimes M_{\nu_1}) \otimes\ldots\otimes (M_{\mu_{N_x}} \otimes M_{\nu_{N_x}})
\end{align}

In [23]:
def boundary_transfer_row(Nx, R1, X, lambdas, M):
    print("calculating E2p ...")
    if R1>=Nx:
        print("Error in def boundary_transfer_row(Nx, R1, lambdas, M): choose R1<Nx")
        return 0
    E2p = np.zeros((2**(2*Nx),2**(2*Nx)))

    # At every lattice site x along the row with length Nx, 
    # the indices mu,nu can take 3 different values mu,nu=(0,1,2).
    # We sum up all the possible (3*3)^Nx configurations contributing to E2p:
    for config in product(range(3),range(3), repeat=Nx):
        munu_of_x = np.array(config) # contains mu(x) at index 2x, nu(x) at 2x+1
        lambda_prod_mu = 1.0
        lambda_prod_nu = 1.0
        XM_prod = X
        
        # loop (product) along a row x:
        for x in range(Nx):
            lambda_prod_mu *= lambdas[munu_of_x[2*x]]   # = \prod_x lambda_{\mu(x)}
            lambda_prod_nu *= lambdas[munu_of_x[2*x+1]] # = \prod_x lambda_{\nu(x)}
            # multiply iteratively all matrices along row:
            # XM_prod = X*(M_{mu_1} \otimes M_{nu_1})...(M_{mu_R1} \otimes M_{nu_R1})*X 
            #           *(M_{mu_{R1+1}} \otimes M_{nu_{R1+1}})...(M_{mu_Nx} \otimes M_{nu_Nx})
            if x == R1-1:
                XM_prod = XM_prod @ np.kron(M[munu_of_x[2*x]], M[munu_of_x[2*x+1]]) @ X
            else:
                XM_prod = XM_prod @ np.kron(M[munu_of_x[2*x]], M[munu_of_x[2*x+1]])

        # calculate tensor product  (M_{mu_1} x M_{nu_1}) x...x (M_{mu_Nx} x M_{nu_Nx})
        # to get matrix form of E2p:
        M_tensor = np.kron(M[munu_of_x[0]], M[munu_of_x[1]])
        for x in range(1,Nx):
            M_tensor = np.kron(M_tensor, np.kron(M[munu_of_x[2*x]], M[munu_of_x[2*x+1]]))
        
        E2p = E2p + (lambda_prod_mu*lambda_prod_nu * np.trace(XM_prod) * M_tensor)
    return E2p

In [24]:
E2p = boundary_transfer_row(4,2,X,lambda_vals,M)

calculating E2p ...


In [25]:
E2p.shape

(256, 256)

Calculate the "bottom" and "top" boundary row 
$$\mathcal X(R_1) = X(x=1) \otimes \ldots \otimes X(x=R_1) \otimes \mathbb{1}(x=R_1+1) \otimes \ldots \otimes \mathbb{1}(x=N_x)$$

In [26]:
def boundary_operator(Nx, R1, X):
    print("calculating XX ...")
    XX = X
    for x in range(1,Nx):
        if x < R1:
            XX = np.kron(XX, X)
        else:
            XX = np.kron(XX, np.identity(4))
    return XX

In [27]:
xx = boundary_operator(4,2,X)
xx.shape

calculating XX ...


(256, 256)

Calculate the full normalized purity for a periodic system of dimensions $N_x \times N_y$ and a subsystem of size $R_1 \times R_2$:

\begin{equation}
\bar p_2 = \frac{Tr[\rho_A^2]}{Tr^2[\rho_A]} = \frac{Tr[\mathcal X(R_1) E^{(2)R_2}_{||}(R_1) \mathcal X(R_1) E^{(2) N_y-R_2}]}{Tr^2[E^{(1) N_y}]}
\end{equation}

In [28]:
def purity(Nx, Ny, R1, R2, X, lambdas, M):
    print("calculating purity ...")
    # matrix rows:
    nrm = norm_rho(Nx, Ny, lambdas, M)
    E2  = double_transfer_row(Nx, lambdas, M)
    E2p = boundary_transfer_row(Nx, R1, X, lambdas, M)
    XX  = boundary_operator(Nx, R1, X)
    
    # overall tiled matrix lattice:
    rho_A_2 = XX @ np.linalg.matrix_power(E2p,R2) @ XX @ np.linalg.matrix_power(E2,Ny-R2)
    
    # normalized purity:
    p2norm = np.trace(rho_A_2)/nrm**2
    
    return p2norm

In [29]:
p2 = purity(4,100,2,60,X,lambda_vals,M)
p2

calculating purity ...
calculating norm ...
calculating E1 ...
calculating E2 ...
calculating E2p ...
calculating XX ...


0.9999987601964608

In [30]:
s2 = -np.log(p2)
s2

1.2398043077229056e-06